In [1]:
using SparseArrays

In [2]:
include("../../src/ocean_model.jl")
include("../../src/schemes/helmholtz_sparse.jl")
include("../../src/schemes/BC.jl")

update_doubly_periodic_bc!

In [ ]:
B

In [3]:
# H_1 = 1.0
# H_2 = 2.0
# M = 4
# P = 4
# zeta = zeros(Float64, M+2, P+2, 2, 3)
# psi = zeros(Float64, M+2, P+2, 2, 3)

# # Initialise the model.
# for z in 1:2
#     a = rand(Float64, (M+2, P+2))
#     zeta[:,:,z,1] = update_ghost_cells(a)
#     b = rand(Float64, (M+2, P+2))
#     psi[:,:,z,1] = update_ghost_cells(b)
# end

# Pm = P_matrix(H_1, H_2)
# P_inv = P_inv_matrix(H_1, H_2)

# zeta_tilde = zeros(Float64, M+2, P+2, 2)
# psi_tilde = zeros(Float64, M+2, P+2, 2)

# for i in 1:2
#     zeta_tilde[:,:,i] = Pm[i,1]*zeta[:,:,1,1] + Pm[i,2]*zeta[:,:,2,1]
#     psi_tilde[:,:,i] = Pm[i,1]*psi[:,:,1,1] + Pm[i,2]*psi[:,:,2,1]
# end

In [4]:
function create_metadata(model::BaroclinicModel)
    DAY = 60*60*24
    
    sample_interval = 1.0*DAY
    sample_timestep = floor(Int, sample_interval / model.dt)

    metadata = Dict("dt" => model.dt,
        "T" => model.T,
        "sample_interval" => sample_interval,
        "sample_timestep" => sample_timestep
    )

    return metadata
end

create_metadata (generic function with 1 method)

In [6]:
function main()
    MINUTES = 60
    DAY = 60*60*24
    KM = 1000
    YEAR = 60*60*24*365

    # Expect to wait 90 days before seeing things.

    H_1 = 1.0*KM
    H_2 = 2.0*KM
    beta = 2*10^-11
    Lx = 4000.0*KM # 4000 km
    Ly = 2000.0*KM # 2000 km
    dt = 15.0*MINUTES # 30 minutes TODO: This needs to be reduced I think for convergence.
    T = 2.0*YEAR
    U = 3.0 # Forcing term of top level.
    M = 256
    dx = Lx / M
    P = Int(Ly / dx)
    visc = 100.0 # Viscosity, 100m^2s^-1
    r = 10^-7 # bottom friction scaler.
    R_d = 60.0*KM # Deformation radious, ~40km. Using 60km for better numerics.
    
    # Create the model.
    model = BaroclinicModel(H_1, H_2, beta, Lx, Ly, dt, T, U, M, P, dx, visc, r, R_d)

    total_steps = floor(Int, T / dt)

    simulation_name = "test15"
    file_name = "../../data/$simulation_name.jld"
    save_sim = false

    zeta, psi = initialise_model(model)

    if save_sim
        metadata = create_metadata(model)
        
        # Create the file and write the initial conditions.
        f = jldopen(file_name, "w") do file
            write(file, "zeta_0", zeta[:,:,:,1])
            write(file, "psi_0", psi[:,:,:,1])
            write(file, "metadata", metadata)
        end
    end 

    println("M = $M, P = $P")
    println("Timestep: $dt")
    println("T: $T")
    println("Total steps: $total_steps")

    sample_interval = 1.0*DAY
    sample_timestep = floor(Int, sample_interval / dt)
    
    poisson_system, helmholtz_system = initialise_linear_systems(model)

    println("Starting timeloop")

    for (timestep, time) in enumerate(1:dt:T)
        zeta = evolve_zeta_top(model, zeta, psi, timestep)
        zeta = evolve_zeta_bottom(model, zeta, psi, timestep)
        psi = evolve_psi(model, zeta, psi, poisson_system, helmholtz_system)

        if timestep % sample_timestep == 0 && save_sim
            t = timestep * time

            println("Timestep: $timestep, time: $t, | ", zeta[10,10,1,1])
            f = jldopen(file_name, "r+") do file
                write(file, "zeta_$timestep", zeta[:,:,:,1])
                write(file, "psi_$timestep", psi[:,:,:,1])
            end
        end
    end
end

main()

M = 256, P = 128
Timestep: 900.0
T: 6.3072e7
Total steps: 70080
Starting timeloop


InterruptException: InterruptException: